In [1]:
#ライブラリインポート
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error
import lightgbm as lgb
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV


from sklearn.externals import joblib
# Pandasでの表示が省略されないように設定
pd.set_option("max_columns", 1000)
pd.set_option('max_rows',1000)
pd.set_option('max_info_columns',1000)

# Jupyterの枠を広げる
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:96%!important;}</style>"))

In [2]:
#全データ読み込み
total_df = pd.read_csv("total_df.csv",index_col=0)
train_df = pd.read_csv("train_df.csv",index_col=0)
test_df = pd.read_csv("test_df.csv",index_col=0)

smpsb_df = pd.read_csv("Data/sample_submit.csv", header=None)

In [3]:
smpsb_df.head()

,0,1
0,19075,17961.112299
1,19076,17961.112299
2,19077,17961.112299
3,19078,17961.112299
4,19079,17961.112299


In [4]:
total_df.head()

,id,section,round,temperature,humidity,match_date_year,match_date_month,match_date_day,match_date_dayofweek,match_date_is_holiday,kick_off_time_hour,kick_off_time_minute,team_名古屋,team_東京V,team_大宮,team_福岡,team_甲府,team_徳島,team_清水,team_川崎F,team_札幌,team_松本,team_広島,team_京都,team_FC東京,team_鳥栖,team_磐田,team_山形,team_仙台,team_浦和,team_C大阪,team_G大阪,team_長崎,team_横浜FC,team_湘南,team_新潟,team_神戸,team_横浜FM,team_大分,team_千葉,team_鹿島,team_柏,important_derby,capacity,broadcasters_NHK総合,home_team_win,home_team_score,away_team_score,weather_晴,weather_曇,weather_雨,weather_屋内,weather_のち,weather_時々,weather_一時,is_sun_sat_hol,join_year,home_team_history,distance,home_rank,away_rank,home_rank_w,away_rank_w
0,9190.0,1.0,1.0,8.3,40.0,2006.0,3.0,4.0,5.0,0.0,16.0,4.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,0.0,21000.0,True,False,1.0,1.0,True,False,False,False,False,False,False,1.0,1992.0,26.0,399.258435,1.0,2.0,1.0,1.0
1,9191.0,1.0,2.0,12.9,28.0,2006.0,3.0,5.0,6.0,0.0,13.0,0.0,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,15859.0,False,False,0.0,2.0,True,False,False,False,False,False,False,1.0,1999.0,19.0,71.435209,-1.0,14.0,1.0,1.0
2,9192.0,1.0,2.0,12.1,35.0,2006.0,3.0,5.0,6.0,0.0,13.0,35.0,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,0.0,48999.0,False,True,2.0,0.0,True,False,False,False,False,False,False,1.0,1999.0,19.0,772.935275,10.0,11.0,1.0,1.0
3,9193.0,1.0,2.0,11.6,42.0,2006.0,3.0,5.0,6.0,0.0,14.0,4.0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,51697.0,False,False,1.0,1.0,True,False,False,False,False,False,False,1.0,1994.0,24.0,704.065706,6.0,-1.0,1.0,1.0
4,9194.0,1.0,2.0,13.1,32.0,2006.0,3.0,5.0,6.0,0.0,14.0,4.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,0.0,20223.0,False,True,3.0,2.0,True,False,False,False,False,False,False,1.0,1992.0,26.0,163.143955,14.0,5.0,1.0,1.0


In [5]:
# 今回は時系列データなので、ある期間との前後でデータを分ける
# 2015年までのデータを訓練データ
# 2016年のデータを検証用データにする
all_train_X = train_df.query("match_date_year <= 2016").drop(["id", "attendance"], axis=1)
all_train_y = train_df.query("match_date_year <= 2016")["attendance"]

train_X = train_df.query("match_date_year <= 2015").drop(["id", "attendance"], axis=1)
train_y = train_df.query("match_date_year <= 2015")["attendance"]

val_X = train_df.query("match_date_year == 2016").drop(["id", "attendance"], axis=1)
val_y = train_df.query("match_date_year == 2016")["attendance"]

test_X = test_df.drop(["id"], axis=1)

In [7]:
filename1 ="Xgboost.sav"
loaded_model_1 = joblib.load(filename1)
filename2 =  "LightGBM.sav"
loaded_model_2 = joblib.load(filename2)
#filename3 = "11_18_Randomforest.sav"
#loaded_model_3 = joblib.load(filename3)

In [8]:
pred1 = loaded_model_1.predict(test_X)

In [10]:
pred2 = loaded_model_2.predict(test_X)

In [13]:
#予測結果をアンサンブル
pred = np.average(np.array([pred1,pred2]),axis=0,weights=[0.55,0.45])
smpsb_df.iloc[:len(test_X), 1]= pred
smpsb_df[1].head()

0    43207.350108
1    15692.499079
2    23482.953121
3    17775.812549
4    19427.452151
Name: 1, dtype: float64

In [17]:
#提出フォーマット作成
smpsb_df.to_csv('result.csv',index=None,header=None)